In [ ]:
import os
#import arcpy
import numpy as np
#from arcpy import env
from pathlib import Path
from os import listdir

class Conversion_DWG_FP(object):
    
    @staticmethod
    def probe_gdb(input_gdb):
        '''
        List names of feature classes. For one class, list all fields names, fields types, and layer names.
        @input_gdb: path to the input GDB file
        '''
        env.workspace = input_gdb
        dataset = arcpy.ListDatasets("*", "Features")[0]
        featureClassList = arcpy.ListFeatureClasses("*", "ALL", dataset)
        print("===== Feature classes:")
        for fc in featureClassList:
            print(fc)
        print("===== Fields details:")
        for fc in featureClassList:
            if fc != "Polyline":
                continue
            fields = arcpy.ListFields(fc)
            for f in fields:
                print(f.name, f.type, f.precision, f.scale, f.length)
        layer_list = []
        print("===== Layer names:")
        for fc in featureClassList:
            if fc != "Polyline":
                continue
            with arcpy.da.SearchCursor(fc, "*") as rows:
                for row in rows:
                    layer_list.append(row[4])
            layer_unique = set(layer_list)
            layer_list = list(layer_unique)
            for lyr in layer_list:
                print(lyr)

    @staticmethod
    def cvt_gdb2dwg(input_gdb, layers, output_path):
        '''
        Note the doors CANNOT yet be drawn correctly in the output dwg files (overlapping circles).
        The problem is explained here: https://support.esri.com/en/technical-article/000013522
        The first attempt is to create an empty feature class and add what is needed to it.
        However, the geometry type cannot be updated/inserted correctly. The legacy codes are retained
        here for future reference. The second attempt is to copy from the original feature class and
        delete what is not needed from it.
        @input_gdb: path to the input GDB file
        @layers: layers needed to output (example: ["A_WALL", "A_DOOR"])
        @output_path: path to the output dwg file
        '''
        field_dict = dict()
        field_dict = {"Blob":"Blob", "Date":"DATE", "Double":"DOUBLE", "Geometry":"Geometry", "GlobalID":"GlobalID",
                      "Guid":"Guid", "Integer":"LONG", "OID":"OID", "Raster":"Raster", "String":"TEXT", "Single":"Single",
                      "SmallInteger":"SHORT"}
        env.workspace = input_gdb
        dataset = arcpy.ListDatasets("*", "All")[0]
        featureClassList = arcpy.ListFeatureClasses("*", "ALL", dataset)

        for fc in featureClassList:
            if fc != "Polyline":
                continue
            fc_cpy = r"in_memory\feature_copy"
            arcpy.CopyFeatures_management(fc, fc_cpy)

            with arcpy.da.UpdateCursor(fc_cpy, "*") as rows:
                for row in rows:
                    if row[4] not in layers:
                        rows.deleteRow()
            arcpy.ExportCAD_conversion(fc_cpy, "DWG_R2010", output_path, "IGNORE_FILENAMES_IN_TABLES", "OVERWRITE_EXISTING_FILES", "")

        # ====== Legacy Codes =====
        # arcpy.DeleteField_management(fc, ["START_X", "START_Y", "START_Z", "MID_X", "MID_Y", "MID_Z", "END_X", "END_Y", "END_Z"])
        # FC = arcpy.CreateFeatureclass_management("in_memory", "FC")
        # arcpy.AddField_management(FC, "layer", "TEXT", field_length = 50)
        # arcpy.AddField_management(FC, "START_X", "DOUBLE")
        # fields = arcpy.ListFields(fc)
        # for i in range(2, len(fields)):
        #     print(fields[i].name, fields[i].type, fields[i].precision, fields[i].scale, fields[i].length)
        #     arcpy.AddField_management(FC, fields[i].name, field_dict[fields[i].type], fields[i].precision, fields[i].scale, fields[i].length)
        # cursor = arcpy.da.InsertCursor(FC, "SHAPE@")
        # with arcpy.da.UpdateCursor(FC, "SHAPE@") as rows:
        #     for row in rows:                             
        #         rows.updateRow(array)
        
    @staticmethod
    def cvt_dwg2gdb(in_folder, out_folder, dataset_name="CAD_features", reference_scale="1000"):
        '''
        Convert dwg files to arcpy compatible gdb files for geometry reading.
        @in_folder: input folder containing dwg files
        @out_folder: output folder to output gdb folders
        @dataset_name: the name of the dataset to which the dwg is converted
        @reference_scale: not sure yet...
        '''
        dwg_list = [f for f in listdir(in_folder) if f.endswith('.dwg')]
        file_num = len(dwg_list)
        Path(out_folder).mkdir(parents=True, exist_ok=True)
        for dwg in dwg_list:
            dwg_name = dwg.split(".")[0]
            dwg_path = os.path.join(in_folder, dwg)
            gdb_path = os.path.join(out_folder, dwg_name+".gdb")
            arcpy.CreateFileGDB_management(out_folder, dwg_name+".gdb")
            arcpy.CADToGeodatabase_conversion(dwg_path, gdb_path, dataset_name, reference_scale)

    @staticmethod
    def extract_geometry_fromGDB(input_gdb, layers, obj_output, expand_layers):
        '''
        Extract layers and read geometry from GDB files. The output is organized as: [rows] * layers
        @input_gdb: path to the input GDB file
        @layers: layers needed to output (example: ["A_WALL", "A_DOOR"])
        @obj_output: if True, the output will be formatted in the way suitable for obj file;
            if False, the output will be formatted for drawing
            The output is now in meters.
        @expand_layers: if True, all layers containing names in argument "layers" will be added,
           if False, only layers in argument "layers" will be used
        @attributes2add: OBSOLETE
            [refer to https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/add-geometry-attributes.htm
            (example: "LENGTH;LINE_START_MID_END")]
        @fields2read: OBSOLETE. [depends on the added attributes, list all fields to find what is needed.
            (example: ['Layer', 'START_X', 'START_Y', 'END_X', 'END_Y'])]
        '''
        env.workspace = input_gdb
        dataset = arcpy.ListDatasets("*", "Features")[0]
        featureClassList = arcpy.ListFeatureClasses("*", "ALL", dataset)
        
        # find all layers containing keywords in argument "layers"
        if expand_layers:
            layer_list = []
            for fc in featureClassList:
                if fc != "Polyline":
                    continue
                with arcpy.da.SearchCursor(fc, "*") as rows:
                    for row in rows:
                        layer_list.append(row[4])
            layer_unique = set(layer_list)
            layer_list = list(layer_unique)
            
            expanded_layers = []
            for i in range(len(layers)):
                for j in range(len(layer_list)):
                    if layers[i] in layer_list[j]:
                        expanded_layers.append(layer_list[j])
            layers = expanded_layers
        # ========================================================

        cdict = {}
        for i in range(len(layers)):
            cdict[layers[i]] = i
        geometry_result = [None] * len(layers)
        for i in range(len(layers)):
            tmp_ = []
            geometry_result[i] = tmp_

        length_unit       = ""
        area_unit         = ""
        coordinate_system = ""

        for fc in featureClassList:
            if fc != "Polyline":
                continue
            #arcpy.AddGeometryAttributes_management(fc, attributes2add, length_unit, area_unit, coordinate_system)
            fields = arcpy.ListFields(fc) # List all fields if necessary: fields[X].name
            error_message_num  = 0
            with arcpy.da.SearchCursor(fc, ["*","SHAPE@"]) as rows:
                for row in rows:
                    if row[4] in layers:
                        if row[-1]:
                            pt_array = row[-1].getPart()
                            if obj_output == True:
                                row_ = [row[4], row[-1].pointCount]
                                for vert in range(row[-1].pointCount):
                                    try:
                                        pt = pt_array.getObject(0).getObject(vert)
                                    except:
                                        error_message_num = error_message_num + 1
                                        break
                                    row_.append(pt.X)
                                    row_.append(pt.Y)
                                if len(row_) >= 6:
                                    row_[1] = int((len(row_) - 2) / 2)
                                    geometry_result[cdict[row[4]]].append(row_)
                            else:
                                for vert in range(row[-1].pointCount-1):
                                    try:
                                        pt0 = pt_array.getObject(0).getObject(vert)
                                        pt1 = pt_array.getObject(0).getObject(vert+1)
                                        row_ = [row[4], pt0.X, pt0.Y, pt1.X, pt1.Y]
                                    except:
                                        error_message_num = error_message_num + 1
                                        break
                                    geometry_result[cdict[row[4]]].append(row_)
        print("Error in getting object from array: %d" %(error_message_num))
        return geometry_result
    
    @staticmethod
    def cvt_geometry_format_obj2drw(geometry):
        '''
        geometry format in obj file: [layer_name, point_num, pt1.X, pt1.Y, ... , ptN.X, ptN.y]
        geometry format for drawing: [layer_name, endpoint1.X, endpoint1.Y, endpoint2.X, endpoint2.Y]
        This function converts from former to latter for obj drawing.
        @geometry: result from FileIO_FP.read_geometry_OBJ or from Conversion_DWG_FP.extract_geometry_fromGDB
          with flag obj_output set as True
        '''
        layer_num = len(geometry)
        geometry_result = [None] * layer_num
        for i in range(layer_num):
            tmp_ = []
            geometry_result[i] = tmp_

        for i in range(layer_num):
            struct_num = len(geometry[i])
            for j in range(struct_num):
                layer_name = geometry[i][j][0]
                point_num  = geometry[i][j][1]
                for k in range(1, point_num):
                    row_ = [layer_name, geometry[i][j][k*2], geometry[i][j][k*2+1],
                            geometry[i][j][(k+1)*2], geometry[i][j][(k+1)*2+1]]
                    geometry_result[i].append(row_)
        return geometry_result
    
    @staticmethod
    def extract_all_points(geometry):
        '''
        This function extracts all points from geometry for hungarian matching
        @geometry: result from FileIO_FP.read_geometry_OBJ or from Conversion_DWG_FP.extract_geometry_fromGDB
          with flag obj_output set as True
        '''
        x_coord = []
        y_coord = []
        layer_num = len(geometry)
        for i in range(layer_num):
            struct_num = len(geometry[i])
            for j in range(struct_num):
                point_num = geometry[i][j][1]
                for k in range(1, point_num+1):
                    x_coord.append(geometry[i][j][k*2])
                    y_coord.append(geometry[i][j][k*2+1])
        return x_coord, y_coord
    
    @staticmethod
    def legacy_for_future_reference():
        path = "E:/Data2/ArcGIS/test03_backup.aprx"
        aprx = arcpy.mp.ArcGISProject(path)
        mp = aprx.listMaps("Map")[0]
        # ====================================================
        layer_group_Annotation = mp.listLayers("testdwg-Annotation Group")[0]
        layer_group_Point = mp.listLayers("testdwg-Point Group")[0]
        layer_group_Polyline = mp.listLayers("testdwg-Polyline Group")[0]
        layer_group_MultiPatch = mp.listLayers("testdwg-MultiPatch Group")[0]
        layer_group_Polygon = mp.listLayers("testdwg-Polygon Group")[0]
        # ====================================================
        layer_group_Annotation.visible = False
        layer_group_Point.visible = False
        layer_group_MultiPatch.visible = False
        layer_group_Polygon.visible = False
        # ====================================================
        # polyline_layers = layer_group_Polyline.listLayers()
        # for layer in polyline_layers:
        #     if layer.name != "A_WALL":
        #         layer.visible = False
        # aprx.save()
        # ====================================================
        polyline_A_WALL = layer_group_Polyline.listLayers("A_WALL")[0]
        md = polyline_A_WALL.metadata
        print(md)